In [1]:
import numpy as np
import pandas as pd

from UpdateDB_ref import Endpoint_annotation as ep


*** CompoundDB module found. Will check the synonyms table to resolve CAS. ***



In [2]:
ep_refactored = ep.Endpoint(host='gea', dbname='cii_v3_test_refactored_2', user='postgres', password='DBAdmin')

In [3]:
ep_pharos = ep.Endpoint(host='localhost', dbname='cii_pharos', user='postgres', password='PSGAdmin')

In [4]:
endpoints_ref = ep_refactored.get_endpoints()

In [5]:
endpoints_pharos = ep_pharos.get_endpoints()

In [6]:
ep_refactored.get_total_annotations_per_endpoint(endpoints_ref)

/home/emarch/miniconda3/envs/inditex/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Endpoints,YES,Pending,NO,No information
0,cmr,1909.0,41.0,890.0,2832.0
1,pbt,87.0,0.0,746.0,4839.0
2,vpvb,97.0,0.0,746.0,4829.0
3,endocrine_disruptor,299.0,0.0,0.0,5373.0
4,sensitiser,133.0,129.0,0.0,2143.0
5,chem_id,0.0,0.0,0.0,0.0


In [7]:
ep_pharos.get_total_annotations_per_endpoint(endpoints_pharos)

/home/emarch/miniconda3/envs/inditex/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Endpoints,YES,Pending,NO,No information
0,subs_id,0.0,0.0,0.0,0.0
1,cmr,1362.0,39.0,477.0,2756.0
2,pbt,38.0,0.0,357.0,4239.0
3,vpvb,43.0,0.0,357.0,4234.0
4,endocrine_disruptor,199.0,0.0,3.0,4432.0
5,sensitiser,64.0,68.0,0.0,1235.0


In [8]:
endpoint_pos = pd.read_sql_query("""select pea.annotation_id, rn.names, pea.endpoint 
from positive_endpoint_annotations pea
left join regulation_names rn on rn.id = pea.annotation_id
order by pea.id """, ep_refactored.conn)

In [9]:
endpoint_dict = {'CMR':endpoint_pos.loc[endpoint_pos['endpoint'] == 'cmr','names'].tolist(),
                 'PBT':endpoint_pos.loc[endpoint_pos['endpoint'] == 'pbt','names'].tolist(),
                 'vPvB':endpoint_pos.loc[endpoint_pos['endpoint'] == 'vpvb','names'].tolist(),
                 'Endocrine Disruptor':endpoint_pos.loc[endpoint_pos['endpoint'] == 'endocrine disruptor','names'].tolist()}

In [10]:
substances = ep_refactored.get_substances().sort_values(by='id').id.unique()

In [11]:
chem_ids = ep_refactored.get_chem_id_dataframe()

In [12]:
chem_ids_unique = chem_ids.loc[chem_ids['chem_type_id'] == 1, 'id'].unique()

In [ ]:
chemid_endpoint = ep_refactored.get_annotations_for_substance(chem_ids_unique, endpoint_dict)

In [14]:
chemid_endpoint.head()

,subs_id,CMR,PBT,vPvB,Endocrine Disruptor,Sensitiser
0,1.0,YES,No information,No information,No information,YES
1,2.0,YES,No information,No information,YES,No information
2,3.0,YES,No information,No information,No information,No information
3,4.0,YES,No information,No information,No information,No information
4,5.0,No information,No information,No information,No information,No information


### Here it should come the check part
After getting the endpoint annotations per substance, and at the same time we update the database with them, the check function should come.
Before updating the DB, the check function should see if there's a substance id in the table.
If no, adds the new substance with the annotations.
If there is, then it checks the annotations and applies the filters we've spoken

In [15]:
# ep.add_annotations_to_database(subs_endpoint)

### Total annotation check
This only counts the total number of annotations per endpoint we have

In [16]:
ep.get_total_annotations_per_endpoint(subs_endpoint)

,Endpoints,YES,Pending,No information
0,CMR,153.0,59.0,1155.0
1,PBT,44.0,0.0,1323.0
2,vPvB,49.0,0.0,1318.0
3,Endocrine Disruptor,148.0,0.0,1219.0
4,Sensitiser,64.0,68.0,1235.0
